# 1a. **CLASSIFICAZIONE**

In [ ]:
# ============================================================
# CELL 1 - IMPORT E SETUP
# ============================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
import seaborn as sns

# ✅ SEED PER RIPRODUCIBILITÀ
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("Librerie importate e Seed impostato.")

In [ ]:
# ============================================================
# CELL 2 - CONFIGURAZIONE
# ============================================================

# Percorsi (Adatta se necessario)
EXCEL_PATH = "/kaggle/input/tumor-segmentation-ai/training_metadata.xlsx"
IMAGES_ROOT = "/kaggle/input/tumor-segmentation-ai/training_images/training_images"

# ⚙️ IPERPARAMETRI OTTIMIZZATI PER EFFICIENTNET-B7
# B7 lavora nativamente a 600x600, ma 300 è un buon compromesso memoria/performance
IMG_SIZE = 300          
BATCH_SIZE = 24          # Abbassato a 8 per evitare OutOfMemory con B7
NUM_WORKERS = 2
NUM_CLASSES = 3

# Mappatura
id2class = {0: "benigno", 1: "maligno", 2: "sano"}
class2id = {v: k for k, v in id2class.items()}

# Device Check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Configurazione: IMG_SIZE={IMG_SIZE}, BATCH={BATCH_SIZE}")
print("Esecuzione su:", device)

In [ ]:
# ============================================================
# CELL 3 - DATA LOADING & LOSS WEIGHTS
# ============================================================

df = pd.read_excel(EXCEL_PATH)
IMG_COL = "US"
LABEL_COL = "LABEL"

# Pulizia
df = df.dropna(subset=[IMG_COL, LABEL_COL]).reset_index(drop=True)
print(f"Dataset caricato: {len(df)} immagini totali.")

# Visualizzazione Distribuzione
counts = df[LABEL_COL].value_counts().sort_index()
print("\nDistribuzione Classi:")
print(counts)

# 🔥 CALCOLO PESI PER LA LOSS (Weighted CrossEntropy)
# Serve per bilanciare le classi (Maligno è meno frequente di Benigno)
# Formula inversa: Peso = 1 / Frequenza (normalizzato)
class_counts = counts.values
total_samples = sum(class_counts)
class_weights = [total_samples / (len(class_counts) * c) for c in class_counts]

# Convertiamo in tensor e spostiamo su GPU
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"\nPesi calcolati per la Loss: {class_weights_tensor.cpu().numpy()}")
# Esempio output atteso: [0.7, 1.3, 1.0] -> Il modello verrà "punito" di più se sbaglia i Maligni

plt.figure(figsize=(5,4))
counts.plot(kind="bar", color=['#4c72b0', '#c44e52', '#55a868'])
plt.xticks(ticks=[0,1,2], labels=[id2class[i] for i in [0,1,2]], rotation=0)
plt.title("Bilanciamento Classi")
plt.grid(axis="y", alpha=0.3)
plt.show()

In [ ]:
# ============================================================
# CELL 4 - DATASET & DATALOADER (FIX: DROP_LAST=TRUE)
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# 1. DEFINIZIONE DATASET
class UltrasoundDataset(Dataset):
    def __init__(self, df, img_col, label_col, root="", transform=None):
        self.df = df.reset_index(drop=True)
        self.img_col = img_col
        self.label_col = label_col
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def _get_img_path(self, idx):
        path_str = str(self.df.loc[idx, self.img_col])
        if self.root:
            return os.path.join(self.root, path_str)
        return path_str

    def __getitem__(self, idx):
        img_path = self._get_img_path(idx)
        label = int(self.df.loc[idx, self.label_col])
        
        try:
            # Carichiamo come RGB
            pil_img = Image.open(img_path).convert("RGB")
            # Convertiamo in Numpy per Albumentations
            img_np = np.array(pil_img)
        except Exception as e:
            print(f"Errore caricamento {img_path}: {e}")
            img_np = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

        if self.transform:
            augmented = self.transform(image=img_np)
            img = augmented['image']
        else:
            img = T.ToTensor()(pil_img)
            
        return img, label

# 2. TRASFORMAZIONI (CLAHE + ELASTIC)
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
    A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.3),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# 3. DATALOADER (CON FIX)
train_df, val_df = train_test_split(df, stratify=df[LABEL_COL], test_size=0.2, random_state=SEED)
print("Train size:", len(train_df), "Val size:", len(val_df))

train_ds = UltrasoundDataset(train_df, IMG_COL, LABEL_COL, IMAGES_ROOT, train_transform)
val_ds = UltrasoundDataset(val_df, IMG_COL, LABEL_COL, IMAGES_ROOT, val_transform)

# 🔥 FIX QUI: drop_last=True
# Scarta l'ultimo batch se è incompleto (es. 2 immagini), evitando che le GPU ne ricevano 1 sola e crashino.
train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS, 
    pin_memory=True, 
    drop_last=True  # <--- FONDAMENTALE PER 2 GPU
)

val_loader = DataLoader(
    val_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS, 
    pin_memory=True, 
    drop_last=False # In validazione non serve (non aggiorniamo i pesi BatchNorm)
)

print("✅ Dataloader corretti (drop_last=True attivato).")

In [ ]:
# ============================================================
# CELL 5 - MODELLO EFFICIENTNET-B7 (FIXED INPLACE ERROR)
# ============================================================

def get_model():
    print("Scaricamento pesi EfficientNet-B7...")
    weights = models.EfficientNet_B7_Weights.DEFAULT
    model = models.efficientnet_b7(weights=weights)

    # Output features di B7 (solitamente 2560)
    in_features = model.classifier[1].in_features
    
    # --- TESTA AVANZATA (Bottleneck) ---
    model.classifier = nn.Sequential(
        # 1. Normalizza le feature in uscita dal backbone
        nn.BatchNorm1d(in_features),
        
        # 2. Layer Intermedio (Bottleneck)
        nn.Linear(in_features, 512),
        nn.ReLU(),
        
        # 3. Dropout (FIX: inplace=False per evitare RuntimeError)
        nn.Dropout(p=0.5, inplace=False), 
        
        # 4. Layer Finale
        nn.Linear(512, NUM_CLASSES)
    )
    
    # Sblocca tutti i parametri
    for param in model.parameters():
        param.requires_grad = True
    
    return model

# 1. Istanzia il modello
model = get_model()

# 2. Configura Dual GPU
if torch.cuda.device_count() > 1:
    print(f"🔥 {torch.cuda.device_count()} GPU rilevate! Attivazione DataParallel.")
    model = nn.DataParallel(model)
else:
    print("⚠️ Una sola GPU rilevata.")

# 3. Sposta su Device
model = model.to(device)

print("Modello B7 con Bottleneck Head pronto (Inplace Error Fixed).")

In [ ]:
# ============================================================
# CELL 6 - TRAINING LOOP COMPLETO (DUAL GPU + LOGGING BATCH)
# ============================================================
import time

# --- 1. DEFINIZIONE FUNZIONI DI SUPPORTO ---
def train_one_epoch(model, loader, criterion, optimizer, epoch_idx, total_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Aggiunto enumerate per avere l'indice del batch
    for i, (inputs, labels) in enumerate(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        # 🟢 PRINT INTERMEDIO OGNI 10 BATCH
        if (i + 1) % 10 == 0:
            print(f"    [Epoca {epoch_idx+1}/{total_epochs}] Batch {i+1}/{len(loader)} -> Loss: {loss.item():.4f}")
        
    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            
    return running_loss / total, correct / total, balanced_accuracy_score(y_true, y_pred)

# --- 2. CONFIGURAZIONE OTTIMIZZATORE PER DUAL GPU ---

# Controllo se il modello è in DataParallel per accedere ai parametri corretti
if hasattr(model, 'module'):
    print("Configurazione parametri per Dual GPU (model.module)...")
    backbone_params = model.module.features.parameters()
    head_params = model.module.classifier.parameters()
else:
    print("Configurazione parametri per Single GPU...")
    backbone_params = model.features.parameters()
    head_params = model.classifier.parameters()

# Differential Learning Rates
optimizer = optim.AdamW([
    {'params': backbone_params, 'lr': 1e-5}, # Backbone lento
    {'params': head_params, 'lr': 1e-3}      # Testa veloce
], weight_decay=1e-2)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

# --- 3. AVVIO TRAINING LOOP ---
NUM_EPOCHS = 60
best_bal_acc = 0
patience_counter = 0
EARLY_STOP_PATIENCE = 15

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_bal_acc': []}

print(f"\n🚀 Avvio Training su {torch.cuda.device_count()} GPU")

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    
    # Train (Passiamo anche l'indice epoca per il print)
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, epoch, NUM_EPOCHS)
    
    # Validation
    val_loss, val_acc, val_bal_acc = validate(model, val_loader, criterion)
    
    # Scheduler Update
    scheduler.step(val_bal_acc)
    
    # Recupero LR corrente
    current_lr_head = optimizer.param_groups[1]['lr']
    
    # Salvataggio storico
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_bal_acc'].append(val_bal_acc)
    
    epoch_time = time.time() - start_time
    
    print(f"🔵 FINE EPOCA {epoch+1}/{NUM_EPOCHS} [{epoch_time:.0f}s] | LR Head: {current_lr_head:.1e}")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}") 
    print(f"   Val Loss:   {val_loss:.4f} | Val Bal Acc: {val_bal_acc:.4f}")
    
    # Checkpoint
    if val_bal_acc > best_bal_acc:
        best_bal_acc = val_bal_acc
        
        # Salvataggio compatibile
        if hasattr(model, 'module'):
            torch.save(model.module.state_dict(), "best_model_b7_dual.pt")
        else:
            torch.save(model.state_dict(), "best_model_b7_dual.pt")
            
        print("   ✓ Miglioramento! Modello salvato.")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"   No improvement ({patience_counter}/{EARLY_STOP_PATIENCE})")
        if patience_counter >= EARLY_STOP_PATIENCE:
            print("   ❌ Early Stopping attivato.")
            break

print(f"\n🏆 Training completato. Best Balanced Acc: {best_bal_acc:.4f}")

In [ ]:
# ============================================================
# CELL 7 - GRAFICI
# ============================================================
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title("Loss During Fine-Tuning")
plt.xlabel("Epoch")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_bal_acc'], label='Val Balanced Acc')
plt.title("Balanced Accuracy During Fine-Tuning")
plt.xlabel("Epoch")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# CELL 8 - VALUTAZIONE DETTAGLIATA
# ============================================================
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 1. Carica il miglior modello salvato
# Ricostruiamo il modello pulito (senza DataParallel wrapper per il caricamento)
model_eval = get_model() 

# Se abbiamo salvato con model.module.state_dict(), le chiavi sono pulite
# Se abbiamo salvato con model.state_dict() e DataParallel, potrebbero avere prefisso 'module.'
state_dict = torch.load("best_model_b7_dual.pt", map_location=device)

# Pulizia chiavi se necessario (rimuove 'module.' se presente)
new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace("module.", "") 
    new_state_dict[name] = v

model_eval.load_state_dict(new_state_dict)
model_eval = model_eval.to(device)
model_eval.eval()

print("Best Model caricato per la valutazione.")

# 2. Predizioni su tutto il Validation Set
y_true, y_pred = [], []
y_probs = []

print("Generazione predizioni...")
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model_eval(inputs)
        
        # Salviamo le probabilità per la ROC curve (opzionale) o TTA
        probs = torch.softmax(outputs, dim=1)
        preds = outputs.argmax(1)
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

# 3. Report Testuale
print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=[id2class[i] for i in range(3)]))

# 4. Matrice di Confusione Grafica
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=id2class.values(), yticklabels=id2class.values())
plt.title(f"Confusion Matrix (Best Bal Acc: {0.8410:.2%})")
plt.xlabel("Predetto")
plt.ylabel("Reale")
plt.show()

In [ ]:
# ============================================================
# CELL 9 - TEST TIME AUGMENTATION (TTA)
# ============================================================

def predict_tta(model, loader):
    model.eval()
    all_probs = []
    all_labels = []
    
    print("Esecuzione TTA (Original + Flip H + Flip V)...")
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            all_labels.extend(labels.cpu().numpy())
            
            # 1. Predizione Originale
            out1 = model(inputs)
            prob1 = torch.softmax(out1, dim=1)
            
            # 2. Predizione Flip Orizzontale
            inputs_h = torch.flip(inputs, dims=[3]) # 3 è l'asse width
            out2 = model(inputs_h)
            prob2 = torch.softmax(out2, dim=1)
            
            # 3. Predizione Flip Verticale
            inputs_v = torch.flip(inputs, dims=[2]) # 2 è l'asse height
            out3 = model(inputs_v)
            prob3 = torch.softmax(out3, dim=1)
            
            # MEDIA DELLE PROBABILITÀ
            avg_prob = (prob1 + prob2 + prob3) / 3.0
            all_probs.extend(avg_prob.cpu().numpy())
            
    return np.array(all_labels), np.array(all_probs)

# Esegui TTA
y_true_tta, y_probs_tta = predict_tta(model_eval, val_loader)
y_pred_tta = np.argmax(y_probs_tta, axis=1)

# Calcola nuova accuracy
tta_acc = balanced_accuracy_score(y_true_tta, y_pred_tta)
print(f"\n🚀 Balanced Accuracy senza TTA: 0.8410")
print(f"🔥 Balanced Accuracy CON TTA:    {tta_acc:.4f}")

# Mostra se è migliorata la matrice
print("\nClassification Report (TTA):")
print(classification_report(y_true_tta, y_pred_tta, target_names=[id2class[i] for i in range(3)]))

# **1b Segementazione**

In [ ]:
# ============================================================
# CELL 1 - SETUP AMBIENTE E LIBRERIE SOTA
# ============================================================
# Installiamo la libreria standard per la segmentazione avanzata
!pip install segmentation_models_pytorch -q

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp

# SEED PER RIPRODUCIBILITÀ TOTALE
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# CONFIGURAZIONE
EXCEL_PATH = "/kaggle/input/tumor-segmentation-ai/training_metadata.xlsx"
IMAGES_ROOT = "/kaggle/input/tumor-segmentation-ai/training_images/training_images"
IMG_SIZE = 256       # Standard aureo per U-Net/MAnet
BATCH_SIZE = 16      # Con 2 GPU T4 va bene (8 img per GPU)
NUM_EPOCHS = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Setup MAnet pronto su {device}.")

In [ ]:
# ============================================================
# CELL 2 - DATA PREPARATION (LEAKAGE-PROOF)
# ============================================================

# 1. Carica il DataFrame COMPLETO (Come nel Notebook 1)
df_full = pd.read_excel(EXCEL_PATH)
df_full = df_full.dropna(subset=['US', 'LABEL', 'MASK']).reset_index(drop=True)

# 2. SPLIT SUL COMPLETO (Stesso Seed = Stesso Split del Classificatore)
# Questo garantisce che train_full e val_full siano identici a quelli del Notebook 1
train_full, val_full = train_test_split(df_full, stratify=df_full['LABEL'], test_size=0.2, random_state=SEED)

print(f"Split Iniziale (Identico al Classificatore):")
print(f"  Train Totale: {len(train_full)}")
print(f"  Val Totale:   {len(val_full)}")

# 3. FILTRO POST-SPLIT (Solo ora rimuoviamo i Normali)
# Manteniamo solo Benign (0) e Malignant (1) DAI RISPETTIVI SET
train_seg = train_full[train_full['LABEL'] != 2].reset_index(drop=True)
val_seg = val_full[val_full['LABEL'] != 2].reset_index(drop=True)

print(f"\nDataset Segmentazione Filtrato (Solo B/M):")
print(f"  Train Seg: {len(train_seg)} (Sicuro: era nel Train del Classificatore)")
print(f"  Val Seg:   {len(val_seg)}   (Sicuro: mai visto da nessuno dei due)")

# 4. Dataset Class (Identica a prima)
class SegmentationDataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.df = df # Non serve reset_index qui perché lo abbiamo fatto sopra
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, str(self.df.loc[idx, "US"]))
        mask_path = os.path.join(self.root, str(self.df.loc[idx, "MASK"]))
        
        try:
            image = np.array(Image.open(img_path).convert("RGB"))
            mask = np.array(Image.open(mask_path).convert("L"))
        except:
            image = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
            mask = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)

        mask = np.where(mask > 0, 1.0, 0.0).astype(np.float32)

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            image = aug['image']
            mask = aug['mask']
            
        mask = mask.unsqueeze(0) 
        return image, mask

# 5. Augmentation (Copia quella di prima)
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.4),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# 6. DATALOADERS (Usiamo i df filtrati post-split)
train_loader = DataLoader(SegmentationDataset(train_seg, IMAGES_ROOT, train_transform), 
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(SegmentationDataset(val_seg, IMAGES_ROOT, val_transform), 
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("✅ Dataloaders pronti. LEAKAGE RISK: 0%.")

In [ ]:
# ============================================================
# CELL 3 - MODELLO MAnet (Multi-scale Attention Net)
# ============================================================

# MAnet è considerata l'erede della Attention U-Net
# Usa EfficientNet-B7 pre-addestrata su ImageNet come Encoder

ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = 1
ACTIVATION = None # Usiamo logits grezzi per stabilità numerica

model = smp.MAnet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels=3, 
    classes=CLASSES, 
    activation=ACTIVATION,
    decoder_pab_channels=64 # Canali per il blocco di attenzione (tuning fine)
)

# Gestione Multi-GPU
if torch.cuda.device_count() > 1:
    print(f"🔥 {torch.cuda.device_count()} GPU attive! DataParallel ON.")
    model = nn.DataParallel(model)

model = model.to(device)

print(f"🚀 Modello MAnet caricato con encoder {ENCODER} pre-addestrato.")

In [ ]:
# ============================================================
# CELL 4 - TRAINING AVANZATO (Con Early Stopping & Batch Monitor)
# ============================================================

# Definizione Loss Ibrida
loss_dice = smp.losses.DiceLoss(mode='binary', from_logits=True)
loss_focal = smp.losses.FocalLoss(mode='binary')

def criterion(pred, target):
    # 40% Dice (Forma) + 60% Focal (Pixel difficili)
    return 0.4 * loss_dice(pred, target) + 0.6 * loss_focal(pred, target)

# Metrica Dice Pura per Valutazione
def dice_metric(pred, target):
    pred = (torch.sigmoid(pred) > 0.5).float()
    intersection = (pred * target).sum()
    return (2. * intersection) / (pred.sum() + target.sum() + 1e-7)

# Optimizer & Scheduler
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
# Scheduler: riduce il LR se ci blocchiamo
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, factor=0.5)

# CONFIGURAZIONE EARLY STOPPING
EARLY_STOP_PATIENCE = 15   # Aspetta 15 epoche senza miglioramenti prima di fermarsi
patience_counter = 0       # Contatore interno

# Loop
best_dice = 0
history = {'train_loss': [], 'train_dice': [], 'val_dice': []}

print(f"🚀 Avvio Training MAnet (Stop dopo {EARLY_STOP_PATIENCE} epoche senza miglioramenti)...")

for epoch in range(NUM_EPOCHS):
    model.train()
    t_loss = 0
    t_dice = 0
    
    # Train Loop con monitoraggio Batch
    for i, (img, mask) in enumerate(train_loader):
        img, mask = img.to(device), mask.to(device)
        
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, mask)
        loss.backward()
        optimizer.step()
        
        loss_val = loss.item()
        dice_val = dice_metric(out, mask).item()
        
        t_loss += loss_val
        t_dice += dice_val
        
        # 🟢 PRINT INTERMEDIO OGNI 10 BATCH
        if (i + 1) % 10 == 0:
            print(f"    [Ep {epoch+1}] Batch {i+1}/{len(train_loader)} | Loss: {loss_val:.4f} | Train Dice: {dice_val:.4f}")
    
    # Calcolo Medie Training
    avg_t_loss = t_loss / len(train_loader)
    avg_t_dice = t_dice / len(train_loader)
    
    # Validation Loop
    model.eval()
    v_dice = 0
    with torch.no_grad():
        for img, mask in val_loader:
            img, mask = img.to(device), mask.to(device)
            out = model(img)
            v_dice += dice_metric(out, mask).item()
            
    avg_v_dice = v_dice / len(val_loader)
    
    # Step dello Scheduler (aggiorna LR se necessario)
    scheduler.step(avg_v_dice)
    
    # Salvataggio Storico
    history['train_loss'].append(avg_t_loss)
    history['train_dice'].append(avg_t_dice)
    history['val_dice'].append(avg_v_dice)
    
    print(f"🔵 FINE EP {epoch+1}/{NUM_EPOCHS} | T. Loss: {avg_t_loss:.4f} | T. Dice: {avg_t_dice:.4f} | Val Dice: {avg_v_dice:.4f}")
    
    # --- LOGICA EARLY STOPPING ---
    if avg_v_dice > best_dice:
        best_dice = avg_v_dice
        # Salvataggio Modello
        save_obj = model.module.state_dict() if hasattr(model, 'module') else model.state_dict()
        torch.save(save_obj, "best_manet_b7.pt")
        print("  ✓ Saved Best MAnet Model (New Record!)")
        patience_counter = 0 # Resetta contatore perché abbiamo migliorato
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{EARLY_STOP_PATIENCE})")
        
        if patience_counter >= EARLY_STOP_PATIENCE:
            print(f"\n❌ EARLY STOPPING ACTIVATED at Epoch {epoch+1}")
            print(f"   Il modello non migliora da {EARLY_STOP_PATIENCE} epoche.")
            break

print(f"🏆 Best Val Dice Score: {best_dice:.4f}")

In [ ]:
# ============================================================
# CELL 5 - VISUALIZZAZIONE RISULTATI
# ============================================================
# Ricarica modello pulito
model = smp.MAnet(encoder_name=ENCODER, encoder_weights=None, classes=CLASSES, activation=ACTIVATION)
model.load_state_dict(torch.load("best_manet_b7.pt"))
model = model.to(device)
model.eval()

img, mask = next(iter(val_loader))
img, mask = img.to(device), mask.to(device)

with torch.no_grad():
    # Sigmoid per probabilità
    pred_logits = model(img)
    pred_mask = (torch.sigmoid(pred_logits) > 0.5).float()

plt.figure(figsize=(12, 10))
for i in range(4): # Mostra 4 casi
    # 1. Immagine
    plt.subplot(4, 3, i*3 + 1)
    im_np = img[i].cpu().permute(1,2,0).numpy()
    im_np = im_np * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
    plt.imshow(np.clip(im_np, 0, 1))
    plt.title("Ultrasound")
    plt.axis('off')
    
    # 2. Ground Truth
    plt.subplot(4, 3, i*3 + 2)
    plt.imshow(mask[i].cpu().squeeze(), cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')
    
    # 3. Predizione MAnet
    plt.subplot(4, 3, i*3 + 3)
    score = dice_metric(pred_logits[i:i+1], mask[i:i+1]).item()
    plt.imshow(pred_mask[i].cpu().squeeze(), cmap='gray')
    plt.title(f"MAnet Prediction (Dice: {score:.2f})")
    plt.axis('off')

plt.tight_layout()
plt.show()

# 1 **Classificazione e segmentazione (B/M)**

In [ ]:
# ============================================================
# PIPELINE FINALE (Custom Head + MAnet)
# ============================================================
!pip install segmentation_models_pytorch -q

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
import segmentation_models_pytorch as smp
import seaborn as sns
from torchvision import models

# CONFIGURAZIONE
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EXCEL_PATH = "/kaggle/input/tumor-segmentation-ai/training_metadata.xlsx"
IMAGES_ROOT = "/kaggle/input/tumor-segmentation-ai/training_images/training_images"

# PATH DEI PESI (I tuoi file caricati)
PATH_CLF = "/kaggle/input/pesi-modelli/best_model_b7_dual (1).pt"
PATH_SEG = "/kaggle/input/pesi-modelli/best_manet_b7.pt"

# RISOLUZIONI (Devono combaciare col training)
IMG_SIZE_CLF = 300 
IMG_SIZE_SEG = 256
NUM_CLASSES = 3
SEED = 42

# MAPPING
id2class = {0: "Benign", 1: "Malignant", 2: "Normal"}

print(f"Pipeline pronta su {DEVICE}.")

In [ ]:
# ============================================================
# CELL 2 - ARCHITETTURE E CARICAMENTO
# ============================================================

# --- 1. CLASSIFICATORE (La tua architettura Custom) ---
def get_classifier():
    # print("Costruzione EfficientNet-B7 con Bottleneck Head...")
    # Pesi a None perché carichiamo i tuoi
    model = models.efficientnet_b7(weights=None) 
    
    # Ricostruzione della testa esatta usata nel training
    in_features = model.classifier[1].in_features
    
    model.classifier = nn.Sequential(
        nn.BatchNorm1d(in_features),
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(p=0.5, inplace=False), 
        nn.Linear(512, NUM_CLASSES)
    )
    return model

# --- 2. SEGMENTATORE (MAnet SOTA) ---
def get_segmentor():
    # Architettura usata nel Notebook 2
    model = smp.MAnet(
        encoder_name="efficientnet-b7",
        encoder_weights=None, 
        in_channels=3,
        classes=1,
        activation=None
    )
    return model

# --- FUNZIONE DI CARICAMENTO SMART ---
def load_weights(model, path):
    if not os.path.exists(path):
        print(f"❌ ERRORE CRITICO: File non trovato: {path}")
        return model
        
    print(f"📂 Caricamento pesi da: {path}")
    state_dict = torch.load(path, map_location=DEVICE)
    
    # Pulizia delle chiavi (rimuove 'module.' se allenato in DataParallel)
    new_state_dict = {}
    for k, v in state_dict.items():
        name = k.replace("module.", "") 
        new_state_dict[name] = v
        
    try:
        model.load_state_dict(new_state_dict)
        print("   -> Pesi caricati correttamente.")
    except Exception as e:
        print(f"   -> ❌ Errore nel caricamento chiavi: {e}")
        
    return model.to(DEVICE).eval()

# --- INIZIALIZZAZIONE ---
print("--- Setup Modelli ---")

clf_model = get_classifier()
clf_model = load_weights(clf_model, PATH_CLF)

seg_model = get_segmentor()
seg_model = load_weights(seg_model, PATH_SEG)

print("\n✅ Entrambi i modelli sono pronti per la pipeline.")

In [ ]:
# ============================================================
# CELL 3 - DATASET PIPELINE
# ============================================================
class PipelineDataset(Dataset):
    def __init__(self, df, root):
        self.df = df.reset_index(drop=True)
        self.root = root
        
        # Per il Classificatore (300x300)
        self.transform_clf = T.Compose([
            T.Resize((IMG_SIZE_CLF, IMG_SIZE_CLF)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        # Per il Segmentatore (256x256)
        self.transform_seg = A.Compose([
            A.Resize(IMG_SIZE_SEG, IMG_SIZE_SEG),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Percorsi
        img_name = str(self.df.loc[idx, "US"])
        mask_name = str(self.df.loc[idx, "MASK"])
        label = int(self.df.loc[idx, "LABEL"])
        
        # Carica Immagine e Maschera
        img_path = os.path.join(self.root, img_name)
        pil_img = Image.open(img_path).convert("RGB")
        np_img = np.array(pil_img)
        
        try:
            mask_path = os.path.join(self.root, mask_name)
            mask = np.array(Image.open(mask_path).convert("L"))
            mask = np.where(mask > 0, 1.0, 0.0).astype(np.float32)
        except:
            # Maschera nera se file non esiste (es. casi Normali)
            mask = np.zeros((np_img.shape[0], np_img.shape[1]), dtype=np.float32)

        # Input CLF
        input_clf = self.transform_clf(pil_img)
        
        # Input SEG (con maschera ridimensionata per valutazione)
        aug = self.transform_seg(image=np_img, mask=mask)
        input_seg = aug['image']
        target_mask = aug['mask'].unsqueeze(0)

        return input_clf, input_seg, label, target_mask, np_img

# Creiamo il Test Set (Stesso seed del training per evitare leakage)
from sklearn.model_selection import train_test_split
df = pd.read_excel(EXCEL_PATH).dropna(subset=["US", "LABEL"])
_, val_df = train_test_split(df, stratify=df["LABEL"], test_size=0.2, random_state=SEED)

test_loader = DataLoader(PipelineDataset(val_df, IMAGES_ROOT), batch_size=1, shuffle=False)
print(f"Dataset Test pronto: {len(val_df)} immagini.")

In [ ]:
# ============================================================
# CELL 5 - REPORT NUMERICO COMPLETO
# ============================================================

# --- 1. CLASSIFICAZIONE ---
print("\n=== RISULTATI TASK 1a (CLASSIFICAZIONE) ===")
# Calcolo Balanced Accuracy
bal_acc = balanced_accuracy_score(results["true_labels"], results["pred_labels"])
print(f"🔥 Balanced Accuracy: {bal_acc:.4f}")

print(classification_report(results["true_labels"], results["pred_labels"],
                            target_names=["Benign", "Malignant", "Normal"]))

# Confusion Matrix
cm = confusion_matrix(results["true_labels"], results["pred_labels"])
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["B", "M", "N"], yticklabels=["B", "M", "N"])
plt.title(f"Confusion Matrix (Bal Acc: {bal_acc:.4f})")
plt.ylabel("Ground Truth")
plt.xlabel("Prediction")
plt.show()

# --- 2. SEGMENTAZIONE ---
df_res = pd.DataFrame({
    "True": results["true_labels"],
    "Pred": results["pred_labels"],
    "Dice": results["dice_scores"]
})

# A. Global Dice (Media su tutto il dataset, inclusi i sani)
dice_global = df_res["Dice"].mean()

# B. Lesion Dice (Media solo sui casi che SONO veramente tumori)
# Include gli 0.0 generati se il classificatore ha perso il tumore.
dice_lesion = df_res[df_res["True"] != 2]["Dice"].mean()

# C. Conditional Dice (Media solo sui tumori che il classificatore ha TROVATO)
# Esclude i falsi negativi. Misura la qualità pura della segmentazione quando si attiva.
dice_tp = df_res[(df_res["True"] != 2) & (df_res["Pred"] != 2)]["Dice"].mean()

print("\n=== RISULTATI TASK 1b (SEGMENTAZIONE) ===")
print(f"1. Global Dice Score (Pipeline Totale):       {dice_global:.4f}")
print(f"2. Lesion Dice Score (Efficacia sui Malati):  {dice_lesion:.4f}")
print(f"3. Conditional Dice (Qualità pura U-Net):     {dice_tp:.4f}")

# Analisi Errori
missed = df_res[(df_res["True"] != 2) & (df_res["Pred"] == 2)]
print(f"\nAnalisi Errori Pipeline:")
print(f"Lesioni perse dal Classificatore (False Negatives): {len(missed)} su {len(df_res[df_res['True']!=2])} totali.")

In [ ]:
# ============================================================
# CELL 6 - VISUALIZZAZIONE GRAFICA
# ============================================================
num_show = 5
plt.figure(figsize=(15, 4*num_show))

for i in range(num_show):
    img = results["images"][i]
    true_m = results["true_masks"][i]
    pred_m = results["pred_masks"][i]
    t_lbl = id2class[results["true_labels"][i]]
    p_lbl = id2class[results["pred_labels"][i]]
    dice = results["dice_scores"][i]
    
    # Colore titolo (Verde=Corretto, Rosso=Sbagliato)
    col = 'green' if t_lbl == p_lbl else 'red'
    
    plt.subplot(num_show, 3, i*3 + 1)
    plt.imshow(img)
    plt.title(f"True: {t_lbl} | Pred: {p_lbl}", color=col, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(num_show, 3, i*3 + 2)
    plt.imshow(true_m, cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')
    
    plt.subplot(num_show, 3, i*3 + 3)
    plt.imshow(pred_m, cmap='gray')
    plt.title(f"Pipeline Mask (Dice: {dice:.2f})")
    plt.axis('off')

plt.tight_layout()
plt.show()